# Introduction to CI Template

## ID Mapping Service Example

This is a demonstratuion how to create a service by wrapping external web API.

In this example, the service worker calls Uniprot ID Mapping service to convert set of IDs into other type.

In [1]:
IP = '192.168.99.100'
BASE = 'http://' + IP + '/v1/'  # You need to change this to your service server

import requests
import json

def jprint(data):
    print(json.dumps(data, indent=4))

# Change this to  your Docker container's IP
HEADERS = {'Content-Type': 'application/json'}

In [2]:
# Show list of available services:

services_url = BASE + 'services'

# Click the link below to see the service list...
print(services_url)

http://192.168.99.100/v1/services


In [3]:
# Show description of the id mapping worker
res = requests.get(BASE + 'services/idmapping')
jprint(res.json())

{
    "portNumber": "5547",
    "parameters": [
        {
            "required": true,
            "name": "ids",
            "type": "array",
            "description": "IDs to be converted"
        },
        {
            "required": true,
            "name": "from",
            "type": "string",
            "description": "Original ID type (e.g., P_ENTREZGENEID)"
        },
        {
            "required": true,
            "name": "to",
            "type": "string",
            "description": "Target ID type (e.g., GENENAME)"
        }
    ],
    "description": "Sample Python worker to convert ID using Uniprot Mapper",
    "serviceName": "idmapping"
}


In [4]:
# Call ID mapping service from client

query1 = {
    "ids": ["B7ZA85", "P12345"],
    "from": "ACC", # From Uniprot ACC
    "to": 'GENENAME' # To human readable gene symbol
}

query2 = {
    "ids": ["kras", "hras"], # Gene names as input
    "from": "GENENAME",
    "to": 'ACC' # To Uniprot ID
}

res1 = requests.post(BASE + 'services/idmapping', data=json.dumps(query1), headers=HEADERS)
res2 = requests.post(BASE + 'services/idmapping', data=json.dumps(query2), headers=HEADERS)

In [5]:
# Check task queue
res = requests.get(BASE + 'queue')
jobs = res.json()
jprint(jobs)

[
    {
        "job_id": "1f3458ae-50db-448d-b415-68d2241daac4",
        "status": "finished"
    },
    {
        "job_id": "58ded7f1-e88f-4b57-809f-fc8e2b83ce3e",
        "status": "finished"
    }
]


In [6]:
# Display the results
for job in jobs:
    
    result_url = BASE + 'queue/' + job['job_id'] + '/result'
    print(result_url)

    res = requests.get(result_url)
    result_str = res.json()

    jprint(result_str)

http://192.168.99.100/v1/queue/1f3458ae-50db-448d-b415-68d2241daac4/result
{
    "hras": [
        "A0A096MNJ8",
        "A0A0D9RF63",
        "A0A0J9YXG8",
        "B2Z7X7",
        "B2Z7X8",
        "B2Z7X9",
        "B2Z7Y0",
        "B2Z7Y1",
        "B2Z7Y2",
        "B2Z7Y3",
        "B2Z7Y4",
        "B2Z7Y5",
        "B2Z7Y6",
        "B3RFG8",
        "B5G0X7",
        "C0H5X4",
        "F5ANT1",
        "F6R9P7",
        "F6TRI4",
        "F6XLE9",
        "F7F5J9",
        "F7F5K9",
        "G1KQH0",
        "G1LVE9",
        "G1RA62",
        "G3MXH2",
        "G3N1U2",
        "G3RHE3",
        "H0UT66",
        "H0XQD9",
        "H2MXD7",
        "H2NCA0",
        "H2Q2V0",
        "H3BGF2",
        "H3BGF3",
        "H9H701",
        "I3LPD9",
        "I3NBL5",
        "J9P550",
        "K7ALD7",
        "K7ETS1",
        "K7G8A2",
        "P01112",
        "P08642",
        "P20171",
        "P78460",
        "Q28EW2",
        "Q4SAD9",
        "Q568K0",
        "Q60529

In [7]:
query_original = {
    "ids": ["brca1", "brca2"], # Gene names as input
    "from": "GENENAME",
    "to": 'ACC' # To Uniprot ID
}

res_original = requests.post(BASE + 'services/idmapping', data=json.dumps(query_original), headers=HEADERS)
job_id_original = res_original.json()["job_id"]

In [8]:
original_result_url = BASE + 'queue/' + job_id_original + '/result'

print(original_result_url)
res = requests.get(original_result_url)

# Extract Uniprot IDs (ACC. number)
result = res.json()
new_query = []
for key in result:
    accs_list = result[key]
    new_query = new_query + accs_list

http://192.168.99.100/v1/queue/f13d2ab7-c450-46ee-bd64-f8c72f226cdd/result


In [9]:
query_next = {
    "ids": new_query[1:20], # send first 20 results
    "from": "ACC",
    "to": "EMBL"
}

res_next = requests.post(BASE + 'services/idmapping', data=json.dumps(query_next), headers=HEADERS)
job_id_next = res_next.json()["job_id"]

In [11]:
next_result_url = BASE + 'queue/' + job_id_next + '/result'

print(next_result_url)
res = requests.get(next_result_url)
jprint(res.json())

http://192.168.99.100/v1/queue/95abdb99-4d65-48c2-a18d-da56cf802f11/result
{
    "A0A068BC92": [
        "AIC83531.1"
    ],
    "A0A068BDV3": [
        "AIC83583.1"
    ],
    "A0A068BDS2": [
        "AIC83533.1"
    ],
    "A0A068BEW3": [
        "AIC83567.1"
    ],
    "A0A068BCA9": [
        "AIC83566.1"
    ],
    "A0A068BCC2": [
        "AIC83586.1"
    ],
    "A0A068BEU0": [
        "AIC83532.1"
    ],
    "A0A068BC94": [
        "AIC83536.1"
    ],
    "A0A068BDT0": [
        "AIC83543.1"
    ],
    "A0A068BEU4": [
        "AIC83537.1"
    ],
    "A0A068BEV8": [
        "AIC83562.1"
    ],
    "A0A068BDU2": [
        "AIC83563.1"
    ],
    "A0A068BFU7": [
        "AIC83535.1"
    ],
    "A0A068BDS6": [
        "AIC83538.1"
    ],
    "A0A068BDU4": [
        "AIC83568.1"
    ],
    "A0A068BFU2": [
        "AIC83530.1"
    ],
    "A0A068BEX2": [
        "AIC83584.1",
        "AIC83587.1",
        "AIC83588.1"
    ],
    "A0A068BCA8": [
        "AIC83561.1"
    ],
    "A0A068BFW6

In [ ]:
# Delete All jobs and results
res = requests.delete(BASE + 'queue')
jprint(res.json())

In [ ]:
# The queue is now empty.
res = requests.get(BASE + 'queue')
jprint(res.json())